# 원, 사각형, 삼각형 구분하기

## 2. 어려운 테스트 문제를 기존 CNN 모델로 푸는 경우

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터셋 생성하기

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


### 모델 정의/설정

In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(24,24,3)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')             
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용할 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 test_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [8]:
model.fit_generator(
    train_generator,
    steps_per_epoch=15,
    epochs=200,
    validation_data=test_generator,
    validation_steps=5
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
15/15 [==============================] - 1s 56ms/step - loss: 1.3222 - accuracy: 0.2845 - val_loss: 1.2490 - val_accuracy: 0.3333
Epoch 2/200
15/15 [==============================] - 0s 20ms/step - loss: 0.2612 - accuracy: 0.9677 - val_loss: 2.8576 - val_accuracy: 0.3333
Epoch 3/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0212 - accuracy: 1.0000 - val_loss: 3.4249 - val_accuracy: 0.4000
Epoch 4/200
15/15 [==============================] - 0s 20ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.4533 - val_accuracy: 0.4000
Epoch 5/200
15/15 [==============================] - 0s 19ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 3.9770 - val_accuracy: 0.4000
Epoch 6/200
15/15 [==============================] - 0s 20ms/step - loss: 2.3953e-04 - accuracy: 1.0000 - val_loss: 4.2455 - val_accuracy: 0.4000
Epoch 7/200
15/15 [==============================] - 0s 20ms/step - loss: 1.2144e-04 - accuracy: 1.0000 - val_loss: 4.3216 - val_accuracy: 0.400

### 모델 평가

In [9]:
# acc = model.evaluate_generator(test_generator, steps=5) - deprecated
acc = model.evaluate(test_generator, steps=5)
print(f'Accuracy: {acc[1]:.4f}')

5/5 [==============================] - 0s 6ms/step - loss: 6.8125 - accuracy: 0.4000
Accuracy: 0.4000
